# Analyzing Population - CIA Factbook with SQL

## Introduction

In this project, we're going to analyze data from the CIA World Factbook (2015), a compendium of statistics about all of the countries on Earth. The Factbook contains various demographic and geographic information. The information we will be focusing the most on in this project will be the population. From there, we will do an in depth analysis regarding the population density and rate of natural increase. 

The SQLite factbook.db database can be downloaded [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db).

## Connecting to the Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of Data

We will start by querying the first 5 rows of all the columns. This will give us an overview of what the chart will look like.

In [2]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


#### Description of Column Names

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- migration_rate – the difference between the number of persons entering (immigrants) and leaving (emigrants) a country during the year, per 1,000 people.

## Statistics Summary

We will take a look at the minimum and maximum of the population in order to determine if there are any outliers that need to be addressed.

In [3]:
%%sql
SELECT  MIN(population) AS min_pop, 
        MAX(population) AS max_pop
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop
0,7256490011


The results ended up being really surprising:
- There is a country with minimum population
- There is a country with a population of 7.2 billion

Neither of these should be the case as there should not be a country without any citizens/residents and there should not be a country that has a population equal to the world population (7.2 billion).

## Analyzing Outliers

In [4]:
%%sql
SELECT name, population
FROM facts
WHERE population == (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0



The 0 population result makes sense since Antarctica was included in the data

In [5]:
%%sql
SELECT name, population
FROM facts
WHERE population == (SELECT MAX(population) 
                     FROM facts);

 * sqlite:///factbook.db
Done.


name,population
World,7256490011



The 7.2 billion number turns out to be the entire World. This is consistent with my statement that there should not be a country population equal to the world population.

## New Statistical Summary 

We will now need to account for the world population and Antarctica. Only then can we see the true values for the maximum and minimum population.

In [6]:
%%sql
SELECT  MIN(population) AS min_pop, 
        MAX(population) AS max_pop
FROM facts
WHERE name<>'World'
AND name<>'Antarctica';

 * sqlite:///factbook.db
Done.


min_pop,max_pop
48,1367485388


## Most Populous

When we think population density, we think of high population. This is why we will take a look to see which country has the highest population and which has the lowest.

In [7]:
%%sql
SELECT name, population, area
FROM facts
WHERE population < (SELECT MAX(population)
                    FROM facts)
AND name<>'European Union'
ORDER BY population DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,area
China,1367485388,9596960
India,1251695584,3287263
United States,321368864,9826675
Indonesia,255993674,1904569
Brazil,204259812,8515770


We can see that China and India have 4 times as many people as the United States (the third on the chart). While these populations are very high, they still both have relatively large amoutns of land (area).

The database had included the European Union and was originally third on the list. However, because it is not an individual country, it was filtered out when queried. 

## Least Populous

In [8]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT MIN(population)
                    FROM facts)
ORDER BY population
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,area
Pitcairn Islands,48,47
Cocos (Keeling) Islands,596,14
Holy See (Vatican City),842,0
Niue,1190,260
Tokelau,1337,12


In our dataset, we have some countries that have a 0 population. In order to remove these, we filtered by querying countries with populations greater than the minimum value (which would be 0). 

While the populations are quite small, 4 out of the 5 countries listed are small islands. As a result, these will most likely not have the highest population density.

## Average Population and Area

Finding the average population and area allows us to have a benchmark for what is considered more densely populated. If the population is above average and the area is below average, then the country is more densely populated than average.

In [9]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
FROM facts
WHERE name<>'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

Population density is defined as the number of people per unit of area. This means that the higher the population and the smaller the area, the more dense it is. Below, we query for above average population and below average area. 

In [10]:
%%sql
SELECT * 
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts 
                    WHERE name <>'World') 
AND area < (SELECT AVG(area)
           FROM facts
           WHERE name<>'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31



Just because the country has an above average population but below average area does not mean it is considered the densest. To calculate this, we would need to divide population by the area.

## Population Density - Highest

In [11]:
%%sql
SELECT name, population, area, ROUND(population/area, 2) AS pop_density
FROM facts
WHERE name<>'World'
ORDER BY pop_density DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,area,pop_density
Macau,592731,28,21168.0
Monaco,30535,2,15267.0
Singapore,5674472,697,8141.0
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.0



The highest population density by far is observed in Macau and Monaco. In general, as we can see, these high values are mostly related to small countries and islands, with the area much below average (555,093). Interestingly enough, the population is also below average (32,242,666 people). This goes to show that even if the population is above average, it does not mean it at the top of the list in terms of population density.

## Population Density - Lowest

In [12]:
%%sql
SELECT name, population, area, ROUND(CAST(population AS Float)/CAST(area AS Float),2) AS pop_density
FROM facts
WHERE pop_density NOT NULL
ORDER BY pop_density
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,area,pop_density
Greenland,57733,2166086,0.03
Svalbard,1872,62045,0.03
Falkland Islands (Islas Malvinas),3361,12173,0.28
Pitcairn Islands,48,47,1.02
Mongolia,2992908,1564116,1.91


The lowest population density are easily Greenland and Svalbard. For Greenland, the contributing factor would be the area that led to its low population density. Svalbard, on the otherhand has a much lower population, and while the area is smaller compared to Greenland, it is still a large enough amount of land to produce such a low population density.

## Observing Rate of Natural Increase

Rate of natural increase is calculated by subtracting the birth rate by the death rate.

First, we will observe both the highest and lowest numbers in terms of birth rates and death rates to see if there are any similarities among the results.

### Birth Rates - Highest

In [13]:
%%sql
SELECT name, birth_rate
FROM facts
ORDER BY birth_rate DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58



The top 10 countries with the highest birth rates are all countries residing in Africa. According to [an article by The Conversation](https://theconversation.com/whats-driving-africas-population-growth-and-what-can-change-it-126362), the reason for these high birth rates are due to a high desire for larger family sizes and low levels of use of modern contraceptives.

### Birth Rates - Lowest

In [14]:
%%sql
SELECT name, birth_rate
FROM facts
WHERE birth_rate NOT NULL
ORDER BY birth_rate
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate
Monaco,6.65
Saint Pierre and Miquelon,7.42
Japan,7.93
Andorra,8.13
"Korea, South",8.19
Singapore,8.27
Slovenia,8.42
Germany,8.47
Taiwan,8.47
San Marino,8.63



Many of the countries on the list are associated with more developed countries. [The National Center for Biotechnology Information](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4255510/) mentions that developed countries have easier access to birth control. Additionally, because children in these societies can be seen as an economic drain caused by housing and education costs, some families choose to forgo child bearing or decide to do so later in life.

### Death Rates - Highest

In [15]:
%%sql
SELECT name, death_rate
FROM facts
ORDER BY death_rate DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8



[The World Population Review](https://worldpopulationreview.com/country-rankings/death-rate-by-country) discusses how these countries listed have high death rates for a variety of reasons. Some countries like Ukraine have poorly financed health care systems while others like countries like Lesotho have high numbers of HIV/AIDS, tuberculosis, and respiratory infections that lead to death. 

According to the chart, these countries tend to come from Eastern Europe and Africa with the exception of Afghanistan.

### Death Rates - Lowest

In [16]:
%%sql
SELECT name, death_rate
FROM facts
WHERE death_rate NOT NULL
ORDER BY death_rate
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,death_rate
Qatar,1.53
United Arab Emirates,1.97
Kuwait,2.18
Bahrain,2.69
Gaza Strip,3.04
Turks and Caicos Islands,3.1
Saudi Arabia,3.33
Oman,3.36
Singapore,3.43
West Bank,3.5


Many countries listed in this chart tend to be from highly developed, wealthy Middle East countries. As a result, they tend to have heavier investments in health care. In addition, [MedicalXpress](https://medicalxpress.com/news/2020-08-infections-deaths-qatar-tackled-covid.html#:~:text=Why%20is%20the%20fatality%20rate,world%2C%20authorities%20and%20experts%20say.) mentions that young population migrant workers moving to these wealthier countries for economic opportunity help keep the fatality rate low.

## Rate of Natural Increase - Highest

In [17]:
%%sql
SELECT name, ROUND(birth_rate - death_rate, 1) AS natural_increase
FROM facts
ORDER BY natural_increase DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,natural_increase
Malawi,33.2
Uganda,33.1
Niger,33.0
Burundi,32.7
Mali,32.1
Burkina Faso,30.3
Zambia,29.5
Ethiopia,29.1
South Sudan,28.7
Tanzania,28.4


From the chart above, we can see that the top 10 countries with the highest rate of natural increase are all in Africa. This is not surprising as the top 10 countries with the highest birth rates were also from Africa. Birth rates and rate of natural increase are directly correlated. These countries were also not at the top when it came to death rates and as such, it is not surprising to see birth rates outpace death rates.

## Rate of Natural Increase - Lowest

In [18]:
%%sql
SELECT name, ROUND(birth_rate - death_rate, 1) AS natural_increase
FROM facts
WHERE natural_increase NOT NULL
ORDER BY natural_increase
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,natural_increase
Bulgaria,-5.5
Serbia,-4.6
Latvia,-4.3
Lithuania,-4.2
Ukraine,-3.7
Hungary,-3.6
Germany,-3.0
Slovenia,-3.0
Romania,-2.8
Belarus,-2.7


Many of the countries listed here indicate a negative natural increase. This means that the population is actually decreasing. Once again, we see many Eastern European countries on this chart with Germany, a Western European country as well. While many of these Eastern European countries were on the high death rate chart which would explain the negative natural increase, Germany who was on the low birth rate chart is also on this chart. If we were to look at Germany's demographics, however, this would not be surprising. 

With Germany's [aging population](https://arc.aarpinternational.org/countries/germany#:~:text=Germany's%20population%20age%2065%20and,about%2041%20million%20by%202050.) where 41% of the germany population is projected to be 65+ by 2050, the combination of low birth rates and higher death rates as a result of an aging population would result in a negative natural increase. 

## Conclusion

In this project, we have analyzed various demographic statistics for all the countries in the world from the CIA World Factbook 2015. Below is a summary of our findings (Top 3) for each category:

- The Largest Population
    - China
    - India
    - United States
- The Smallest Population
    - Pitcairn Islands
    - Cocos (Keeling) Islands
    - Holy See (Vatican City)
- The Highest Population Density
    - Macau
    - Monaco
    - Singapore
- The Lowest Population Density
    - Greenland
    - Svalbard
    - Falkland Islands (Islas Malvinas)
- The Highest Birth Rates
    - Niger
    - Mali
    - Uganda
- The Lowest Birth Rates
    - Monaco
    - Saint Pierre and Miquelon
    - Japan
- The Highest Death Rates
    - Lesotho
    - Ukraine
    - Bulgaria
- The Lowest Death Rates
    - Qatar
    - United Arab Emirates
    - Kuwait
- The Largest Natural Increase Rate
    - Malawi
    - Uganda
    - Niger
- The Smallest Natural Increase Rate
    - Bulgaria
    - Serbia
    - Latvia

#### Findings:
- African countries tended to have high birth and death rates. Despite the high death rates, the much higher birth rates result in high natural increase rates
- Eastern European countries tended to have high death rates which would sometimes result in small natural increase rates. 
- Middle East countries (Developed) have the lowest death rates as a result of a heavy investments in healthcare.
- Macau and Monaco have such small area with such a large population that they have the highest (5-figure) population density.
- Greenland and Svalbard have such large areas with a small population that results in having population density in the low hundreths place (0.03).